# Predicting Game Newsletter Subscription Using K-Nearest Neighbors

## Introduction
The purpose of this data science project is to analyze and predict player behaviours on a Minecraft server based on data collected by a UBC Computer Science research group. This data consisted of two primary files: players.csv and sessions.csv. The players.csv file included detailed information about each player, such as their age, experience level, email, and other personal traits. The sessions.csv file tracked individual play sessions for each player based on their email, including data on session times. The specific question we addressed in this project is: Do players' age and experience level corellate to subscription status?

By analyzing both the age and the experience level of a player, we will build a predictive model (K-NN Classification) to assess the likelihood of player subscribing to the newsletter based on such characteristics.


To answer this question, we focus solely on using the players.csv dataset file, as it includes the relevant characteristics (age and experience level) we will be using to hypothesize a player's subscription behaviour. This will involve exploring the dataset to understand the relationships between these variables and subscription status, followed by the development and assessment of the accuracy of our model to form predictions. 



### Research Question

**What player characteristics and behaviors are most predictive of subscribing to a game-related newsletter, and how do these features differ between various player types?**

### Dataset Description

The dataset contains 196 observations with the following variables:

- `experience`: Player experience level (Pro, Veteran, Regular, Amateur)
- `subscribe`: Whether the player subscribed to the newsletter
- `played_hours`: Total hours played
- `Age`: Player's age
- `gender`: Player's gender
- `hashedEmail` and `name`: Removed for irrelevance

This analysis uses techniques covered in the DSCI 100 course including data wrangling, visualization, and K-Nearest Neighbors classification.


**Our goal was to predict whether a player would subscribe to a game newsletter based on a set of player's feature. Since the target variable (subscribe) is binary (TRUE/FALSE), we framed this as a classification problem.**

**We selected the K-Nearest Neighbors (KNN) algorithm because it is non-parametric, easy to understand, and directly influenced by the similarity between features. This cooperate well with our small dataset and helps build an easy understanding of how different features impact prediction outcomes.**

**Because our predictors are categorical variables, we performed one-hot encoding to convert them into a numerical format to be used for KNN algorithm.**

**Before modeling, we prepared the dataset as follows:**

Removed unnecessary variables (name, hashedEmail)
Converted categorical variables into numeric values:
experience: Pro = 3, Veteran = 2, Regular = 1, Amateur = 0
gender: Male = 1, Others = 0
Transformed logical subscribe column to numeric (1 or 0)
Removed rows with missing values using drop_na()

We summarized numeric features to understand their scale and variance (e.g., max, min, mean). We visualized four relationships:

- Subscription Rate by Experience Level

- Played Hours vs Subscription

- Age Distribution vs Subscription

- Gender vs Subscription Rate

In [ ]:
library(tidyverse)
library(class)
library(rsample)
library(purrr)





In [ ]:

players <- read_csv("players.csv")

players_clean <- players |>
  select(-hashedEmail, -name) |>
  mutate(
    subscribe = as.numeric(subscribe),
    gender = ifelse(gender == "Male", 1, 0),
    experience = case_when(
      experience == "Pro" ~ 3,
      experience == "Veteran" ~ 2,
      experience == "Regular" ~ 1,
      experience == "Amateur" ~ 0,
      TRUE ~ NA_real_)
  ) |>
  drop_na()

head(players_clean)


In [ ]:

players_clean |> 
  summarise(
    max_hours = max(played_hours),
    min_hours = min(played_hours),
    mean_hours = mean(played_hours),
    max_age = max(Age),
    min_age = min(Age),
    mean_age = mean(Age)
  )


In [ ]:
ggplot(players, aes(x = experience, fill = factor(subscribe))) +
  geom_bar(position = "dodge") +
  labs(title = "Figure 1. Subscription Rate by Experience Level", x = "Experience", y = "Count", fill = "Subscribed")


In [ ]:
ggplot(players, aes(x = factor(subscribe), y = played_hours)) +
  geom_boxplot() +
  labs(
    title = "Figure 2. Played Hours vs Subscription",
    x = "Subscribed",
    y = "Played Hours"
  )


In [ ]:
ggplot(players, aes(x = Age, fill = factor(subscribe))) +
  geom_histogram(bins = 15, alpha = 0.6, position = "identity", color = "black") +
  labs(title = "Figure 3. Age Distribution by Subscription", x = "Age", y = "Count", fill = "Subscribed")

In [ ]:
ggplot(players, aes(x = gender, fill = factor(subscribe))) +
  geom_bar(position = "fill") +
  labs(
    title = "Figure 4. Subscription Proportion by Gender",
    x = "Gender",
    y = "Proportion",
    fill = "Subscribed"
  )

### **Findings on Player Type Differences**  

Based on the visualizations generated, we can analyze the vary in characteristics and how these factors influence **subscription behaviour**.



#### **1 Experience Level vs Subscription Rate**
- Pro and Veteran players have the highest subscription rates, while Regular and Amateur players have lower subscription rates.

#### **2 Played Hours vs Subscription**
- Subscribed players tend to have higher median played hours.


#### **3 Age vs Subscription**
- Younger players are more likely to subscribe, at same time older players (above 30) show a lower subscription rate.

- Older players may focus on the playing alone


#### **4 Gender vs Subscription**
- Male players have a slightly higher subscription rate than female players, but overall, gender plays a slight effect on subscription behavior.

In [ ]:
set.seed(123)

train <- initial_split(cancer, prop = 0.80, strata = Class)
cancer_train <- training(cancer_split)
cancer_test <- testing(cancer_split)

train <- initial_split()




train_X <- train |> 
  select(played_hours, Age, experience, gender) |> 
  mutate(across(everything(), scale))


test_X <- test |> 
  select(played_hours, Age, experience, gender) |> 
  mutate(across(everything(), scale))





We split the dataset into training and test sets using an 80/20 ratio in initial_split(). We used the scale() function to normalize numerical predictors to ensure they are on comparable scales, which is essential for KNN performance.

In [ ]:


train_X <- train |> select(played_hours, Age, experience, gender)
train_y <- train |> pull(subscribe)
test_X <- test |> select(played_hours, Age, experience, gender)
test_y <- test |> pull(subscribe)

In [ ]:

accuracies <- c()


for (k in 1:20) {
  prediction <- knn(train_X, test_X, cl = train_y, k = k)
  accuracy <- mean(prediction == test_y)
  accuracies <- c(accuracies, accuracy)
}


accuracy_df <- tibble(
  k = 1:20,
  accuracy = accuracies
)


ggplot(accuracy_df, aes(x = k, y = accuracy)) +
  geom_line(color = "blue") +
  labs(
    title = "Figure 5. KNN Accuracy vs K",
    x = "Number of Neighbors (K)",
    y = "Prediction Accuracy"
  )


best_k <- accuracy_df |> 
  filter(accuracy == max(accuracy)) |> 
  pull(k)

best_k


We tested K values from 1 to 20 and calculated test accuracy for each value. This was implemented using a loop and knn() from the package. The result was plotted as Figure 5, showing how model performance varies with K.


We selected the value of 20 as the best k value.

## Discussion
**Figure 1: Subscription Rate by Experience Level**

Analysis:
The plot clearly shows that players with higher experience levels (Veteran and Pro) have higher subscription rates than Regular or Amateur players. This supports the assumption that more experienced players are more engaged with the game and thus more likely to subscribe. What's more, it may also reflect that experienced players have been in the game longer and had more possibility to see the newsletter option.

 **Figure 2: Played Hours vs Subscription (Boxplot)**

Analysis :
The boxplot indicates that subscribed players tend to have higher median and maximum playtime. This seems intuitive—more invested players may want to stay updated. However, it’s also possible that subscription leads to more engagement over time (reverse causality)

**Figure 3:Age Distribution by Subscription**

Analysis :
Younger players appear more likely to subscribe. This may reflect digital habits: younger users tend to be more open to email subscriptions, and more accustomed to in-game notifications and marketing. Oppoesitely, it may be that older users are less willing o accpect                                                                 **Figure 4: Subscription Count by Gender**

Analysis :
There is a small difference in subscription counts between male and non-male players. However, the difference is minor and may not be statistically meaningful. 

These results mostly align with our expectations. It’s true that highly engaged players—those who play longer or identify as experienced—are more likely to subscribe. These players may have a greater emotional investment in the game, and thus are more interested in receiving updates and special content. The limited effect of gender was also expected, as gender is often a weak predictor unless the product is gender-targeted.

These findings can help game developers better understand user behavior. For example, promotional efforts could be tailored to high-playtime players to increase conversions. Knowing that experience and age influence subscription likelihood, marketing emails could highlight tips, achievements, or community events to attract younger or more engaged players.Furthermore,The game team could also consider promoting newsletter subscriptions through platforms like Instagram and TikTok, which are more popular among younger audiences.

## Project Limitations and Improvements
**Sample Size**: With only 196 players in the dataset, the sample may not be sufficiently representative of the overall player population. In particular, certain groups of people (e.g., older players, non-male genders, or very casual users) may be underrepresented, potentially biasing the model’s conclusions. Expanding the dataset with more diverse and balanced samples would improve the robustness and generalizability of the findings.

**Feature Limitation**: The model currently relies on a limited set of variables: age, gender, experience, and total playtime. These features, while relevant, provide only a static snapshot of user behavior. Incorporating dynamic behavioral indicators such as login frequency, session duration, recent activity, or interaction with in-game content could significantly enhance predictive power. Behavioral features often capture intent and engagement more effectively than static demographics.

**Platform Bias**: The dataset does not capture from which channels players encountered or accessed the subscription option. Players who engage primarily via mobile or social platforms might behave differently. Future data collection could include interaction channels to analyze cross-platform behavior differences.